In [9]:
import numpy as np
from pose_format import Pose

In [10]:
pose_file_1 = '../data/pose_files/SGB_FSS_original.pose'
save_path = "../output/08_advanced_features/"

In [11]:

def load_pose(path: str) -> Pose:
    with open(path, 'rb') as f:
        return Pose.read(f.read())

def save_as_pose(pose: Pose, output_path: str):
    """
    Saves a Pose object to a .pose file.
    """
    with open(output_path, 'wb') as f:
        pose.write(f)
    print(f"✅ Saved to: {output_path}")

In [12]:
pose = load_pose(pose_file_1)
save_as_pose(pose, save_path + "loaded_pose.pose")

✅ Saved to: ../output/08_advanced_features/loaded_pose.pose


## 1️⃣ Cropping and Component Selection

In [13]:
def get_specific_components(pose: Pose, components: list) -> Pose:
    """
    Extract specific components
    
    Parameters:
    -----------
    components : list
        List of component names
    """
    filtered = pose.get_components(components)
    
    print(f"✅ Extracted components: {components}")
    print(f"   Original points count: {pose.body.data.shape[2]}")
    print(f"   New points count: {filtered.body.data.shape[2]}")
    
    return filtered

In [14]:
# Example: Hands only
hands_only = get_specific_components(pose, ['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS'])
save_as_pose(hands_only, save_path + "hands_only.pose")
# Example: Body and Face
body_face = get_specific_components(pose, ['POSE_LANDMARKS', 'FACE_LANDMARKS'])
save_as_pose(body_face, save_path + "body_and_face.pose")

✅ Extracted components: ['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS']
   Original points count: 203
   New points count: 42
✅ Saved to: ../output/08_advanced_features/hands_only.pose
✅ Extracted components: ['POSE_LANDMARKS', 'FACE_LANDMARKS']
   Original points count: 203
   New points count: 161
✅ Saved to: ../output/08_advanced_features/body_and_face.pose


In [15]:
def get_specific_points(pose: Pose, components: list, points_dict: dict) -> Pose:
    """
    Extract specific points from specific components
    
    Parameters:
    -----------
    components : list
        List of component names
    points_dict : dict
        Dictionary: component name -> list of point names
    """
    filtered = pose.get_components(components, points=points_dict)
    
    print(f"✅ Extracted specific points")
    return filtered

# Example: Only specific points from the body
upper_body = get_specific_points(
    pose,
    ['POSE_LANDMARKS'],
    {
        'POSE_LANDMARKS': [
            'NOSE', 'LEFT_SHOULDER', 'RIGHT_SHOULDER',
            'LEFT_ELBOW', 'RIGHT_ELBOW',
            'LEFT_WRIST', 'RIGHT_WRIST',
        ]
    }
)
save_as_pose(upper_body, save_path + "upper_body.pose")

✅ Extracted specific points
✅ Saved to: ../output/08_advanced_features/upper_body.pose


In [16]:
def remove_components(pose: Pose, components_to_remove: list) -> Pose:
    """
    Remove components from the pose
    """
    filtered = pose.remove_components(components_to_remove)
    
    print(f"✅ Removed components: {components_to_remove}")
    return filtered

# Example: Remove POSE_WORLD_LANDMARKS
pose_no_world = remove_components(pose, ['POSE_WORLD_LANDMARKS'])
save_as_pose(pose_no_world, save_path + "pose_no_world.pose")

✅ Removed components: ['POSE_WORLD_LANDMARKS']
✅ Saved to: ../output/08_advanced_features/pose_no_world.pose


## 2️⃣ Computing Bounding Box

🎯 **What is a Bounding Box?**

A Bounding Box is the rectangle that surrounds a set of points.


TOP_LEFT ●─────────────┐
            │             │
            │   Points    │
            │      ●  ● ● │
            │     ● ●  ●  │
            └─────────────● BOTTOM_RIGHT


In [17]:
def compute_bounding_box(pose: Pose) -> Pose:
    """
    Compute bounding box for each component
    
    Returns two points for each component:
    - TOP_LEFT: Upper left corner
    - BOTTOM_RIGHT: Lower right corner
    """
    bbox_pose = pose.bbox()
    
    print("✅ Bounding Box computed")
    print(f"   Original shape: {pose.body.data.shape}")
    print(f"   BBox shape: {bbox_pose.body.data.shape}")
    
    return bbox_pose

bbox = compute_bounding_box(pose)

save_as_pose(bbox, save_path + "bbox.pose")

✅ Bounding Box computed
   Original shape: (133, 1, 203, 3)
   BBox shape: (133, 1, 8, 3)
✅ Saved to: ../output/08_advanced_features/bbox.pose


In [18]:
def get_hands_bounding_box(pose: Pose) -> dict:
    """
    Get bounding box for hands only
    """
    # Extract hands
    hands = pose.get_components(['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS'])
    
    # Calculate bbox
    bbox = hands.bbox()
    
    # Extract coordinates
    data = bbox.body.data
    
    result = {}
    for frame_idx in range(data.shape[0]):
        # Left hand: points 0-1, Right hand: points 2-3
        left_tl = data[frame_idx, 0, 0, :2]  # Top-Left
        left_br = data[frame_idx, 0, 1, :2]  # Bottom-Right
        right_tl = data[frame_idx, 0, 2, :2]
        right_br = data[frame_idx, 0, 3, :2]
        
        result[frame_idx] = {
            'left_hand': {'top_left': left_tl, 'bottom_right': left_br},
            'right_hand': {'top_left': right_tl, 'bottom_right': right_br}
        }
    
    return result

boxes = get_hands_bounding_box(pose)

### 2.1 Practical Uses:

**Use Case 1: Track hand position**

In [19]:

boxes = get_hands_bounding_box(pose)

# Frame 10
frame_10 = boxes[10]
print(f"Left hand at: {frame_10['left_hand']['top_left']}")
print(f"Right hand at: {frame_10['right_hand']['top_left']}")

# Useful to know where hands are in each frame

Left hand at: [284.1441650390625 400.3851013183594]
Right hand at: [239.14031982421875 327.1969909667969]


**Use Case 2: Detect hand crossing**

In [20]:

def hands_are_crossing(bbox_frame):
    """Are the hands crossing?"""
    left = bbox_frame['left_hand']
    right = bbox_frame['right_hand']
    
    # Check for intersection
    if (left['bottom_right'][0] > right['top_left'][0] and
        left['top_left'][0] < right['bottom_right'][0]):
        return True
    return False

# Check all frames
boxes = get_hands_bounding_box(pose)
for frame_idx, bbox in boxes.items():
    if hands_are_crossing(bbox):
        print(f"Frame {frame_idx}: Hands are crossing!")
        
# Useful for signs like "X" or "prayer"

Frame 0: Hands are crossing!
Frame 1: Hands are crossing!
Frame 6: Hands are crossing!
Frame 7: Hands are crossing!
Frame 8: Hands are crossing!
Frame 9: Hands are crossing!
Frame 10: Hands are crossing!
Frame 11: Hands are crossing!
Frame 12: Hands are crossing!
Frame 13: Hands are crossing!
Frame 15: Hands are crossing!
Frame 16: Hands are crossing!
Frame 17: Hands are crossing!
Frame 18: Hands are crossing!
Frame 19: Hands are crossing!
Frame 99: Hands are crossing!
Frame 100: Hands are crossing!
Frame 102: Hands are crossing!
Frame 103: Hands are crossing!
Frame 104: Hands are crossing!
Frame 105: Hands are crossing!
Frame 106: Hands are crossing!
Frame 107: Hands are crossing!
Frame 108: Hands are crossing!
Frame 113: Hands are crossing!
Frame 114: Hands are crossing!
Frame 115: Hands are crossing!
Frame 116: Hands are crossing!
Frame 117: Hands are crossing!
Frame 118: Hands are crossing!
Frame 119: Hands are crossing!
Frame 120: Hands are crossing!
Frame 121: Hands are crossing!

## 3️⃣ Backend Conversion

In [21]:
def convert_to_torch(pose: Pose):
    """
    Convert to PyTorch tensors
    """
    torch_pose = pose.torch()
    
    print(f"✅ Converted to PyTorch")
    print(f"   Data type: {type(torch_pose.body.data)}")
    
    return torch_pose

torch_pose = convert_to_torch(pose)

✅ Converted to PyTorch
   Data type: <class 'pose_format.torch.masked.tensor.MaskedTensor'>


In [22]:
def convert_to_tensorflow(pose: Pose):
    """
    Convert to TensorFlow tensors
    """
    tf_pose = pose.tensorflow()
    
    print(f"✅ Converted to TensorFlow")
    print(f"   Data type: {type(tf_pose.body.data)}")
    
    return tf_pose

# tf_pose = convert_to_tensorflow(pose)

## 4️⃣ Additional Helper Functions

In [23]:
def focus_pose(pose: Pose) -> Pose:
    """
    Adjust pose to start from (0,0) and fill available space
    Useful for display and visualization
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    print("✅ Pose adjusted (focus)")
    return pose_copy

focused = focus_pose(pose)
save_as_pose(focused, save_path + "focused.pose")

✅ Pose adjusted (focus)
✅ Saved to: ../output/08_advanced_features/focused.pose


In [24]:
def slice_pose(pose: Pose, start: int = 0, end: int = None, step: int = 1) -> Pose:
    """
    Slice a portion of the pose
    
    Parameters:
    -----------
    start : int
        Start frame
    end : int
        End frame (None = last frame)
    step : int
        Step (1 = every frame, 2 = every 2nd frame, ...)
    """
    pose_copy = pose.copy()
    
    if end is None:
        end = pose_copy.body.data.shape[0]
    
    # Slice the body data directly
    pose_copy.body.data = pose_copy.body.data[start:end:step]
    
    print(f"✅ Pose sliced")
    print(f"   From frame {start} to {end} with step {step}")
    print(f"   Frames: {pose.body.data.shape[0]} -> {pose_copy.body.data.shape[0]}")
    
    return pose_copy

# First 100 frames
first_100 = slice_pose(pose, start=0, end=100)
save_as_pose(first_100, save_path + "first_100.pose")

# Every 2nd frame
every_second = slice_pose(pose, step=2)
save_as_pose(every_second, save_path + "every_second.pose")

✅ Pose sliced
   From frame 0 to 100 with step 1
   Frames: 133 -> 100
✅ Saved to: ../output/08_advanced_features/first_100.pose
✅ Pose sliced
   From frame 0 to 133 with step 2
   Frames: 133 -> 67
✅ Saved to: ../output/08_advanced_features/every_second.pose


## 5️⃣ Data Flattening

**What is Flattening?**

Flattening = Converting multi-dimensional data into 2D format (rows and columns)

**Before Flattening:**
```python
# Data shape: [frames, people, points, dimensions]
#            [100, 1, 75, 3]

pose = load_pose('sign.pose')
print(pose.body.data.shape)  # [100, 1, 75, 3]
```

**After Flattening:**
```python
# One row per point per frame
flat = flatten_pose_data(pose)
print(flat.shape)  # [N, 7]

# Columns: [frame, person, point, confidence, x, y, z]
```

In [25]:
def flatten_pose_data(pose: Pose) -> np.ndarray:
    """
    Flatten data for ML usage
    Removes points with zero confidence
    """
    flat = pose.body.flatten()
    
    print(f"✅ Data flattened")
    print(f"   Shape: {flat.shape}")
    print(f"   Columns: [frame, person, point, confidence, x, y, z]")
    
    return flat

flat_data = flatten_pose_data(pose)
print(flat_data[:5]) # Display first 5 rows of flattened data

✅ Data flattened
   Shape: (26705, 7)
   Columns: [frame, person, point, confidence, x, y, z]
[[  0.           0.           0.           0.99545729 331.33071899
  120.33316803  -0.83765996]
 [  0.           0.           1.           0.98735136 343.02304077
  104.05989838  -0.78138584]
 [  0.           0.           2.           0.98721689 350.11105347
  104.8104248   -0.78215271]
 [  0.           0.           3.           0.98832768 359.10119629
  106.03274536  -0.78219563]
 [  0.           0.           4.           0.99146909 319.21643066
  103.28619385  -0.78185833]]


### 5.1 Example - Export to CSV:

In [26]:

import pandas as pd

# Flatten the data
flat = flatten_pose_data(pose)

# Convert to DataFrame
df = pd.DataFrame(flat, columns=[
    'frame', 'person', 'point', 'confidence', 'x', 'y', 'z'
])

# Save as CSV
df.to_csv(save_path + 'pose_data.csv', index=False)

✅ Data flattened
   Shape: (26705, 7)
   Columns: [frame, person, point, confidence, x, y, z]


## 6️⃣ Features Summary

| Feature | Function | Usage |
|---------|----------|-------|
| Cropping Components | `pose.get_components()` | Extract hands only, face, etc. |
| Remove Components | `pose.remove_components()` | Delete unnecessary components |
| Bounding Box | `pose.bbox()` | Calculate component bounds |
| Focus Pose | `pose.focus()` | Center and scale pose |
| Slice Pose | `pose.body.data[start:end:step]` | Extract frame range |
| Flatten Data | `pose.body.flatten()` | Convert to 2D array for ML |
| Backend Conversion | `pose.torch()` / `pose.tensorflow()` | Convert for training |

### Key Operations Covered
1. **Component Selection**: Extract or remove specific body parts
2. **Bounding Box**: Calculate spatial boundaries for components
3. **Data Transformation**: Flatten for ML, slice for frame selection
4. **Backend Support**: Convert to PyTorch or TensorFlow formats

### Best Practices
- Use `get_components()` to reduce data dimensionality
- Apply `bbox()` for spatial analysis and tracking
- Choose appropriate backend for your deep learning framework